### Base chatbot

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}
Answer: Let's think step by step."""

OLLAMA_DEFAULT_MODEL = os.getenv("OLLAMA_DEFAULT_MODEL")
prompt = ChatPromptTemplate.from_template(template=template)
model = OllamaLLM(model=OLLAMA_DEFAULT_MODEL, temperature=0)

chain = prompt | model
response = chain.invoke({"question": "What is 2+2"})
print(response)

Step 1: Identify the question and understand what it is asking for. In this case, the question is about performing a basic arithmetic operation - addition. The question asks us to find the sum of two numbers, which are 2 and 2.

Step 2: Recall or recall previously learned information. We know that in mathematics, when we add two numbers together, we simply need to combine them. 

Step 3: Perform the operation. Now, let's perform the addition operation on the numbers 2 and 2. 

2 + 2 = 4

So, the answer is 4.


### Retrieve agent

In [2]:
from qdrant_client import QdrantClient

if os.getenv("IS_DOCKER"):
    qdrant_host = "localhost"
else:
    qdrant_host = os.getenv("QDRANT_HOST")
    
qdrant_url = f"http://{qdrant_host}:{os.getenv('QDRANT_PORT')}"
client = QdrantClient(url=qdrant_url)


In [3]:
from langchain_ollama.embeddings import OllamaEmbeddings

user_query = "why emdee"
ollama_embedding = OllamaEmbeddings(
    model=os.getenv("OLLAMA_EMBEDDING_MODEL"),
    temperature=0
)

embedding = ollama_embedding.embed_query(user_query)

In [4]:
result = client.query_points(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    query=embedding,
    limit=10
)

In [5]:
for point in result.points:
    print(point.payload)

{'h1': 'OPERATIONAL HOUR', 'h2': 'Bagaimana jika ingin melakukan pembatalan booking?', 'h3': 'FAQ', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'WHY EMDEE?', 'h3': 'MISI', 'h4': 'OUR JOURNEY', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'Solusi terbaik mengatasi permasalahan kulit wajahmu', 'h3': 'Purifying Wash', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'Solusi terbaik mengatasi permasalahan kulit wajahmu', 'h3': 'Purifying Cleanser', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'WHY EMDEE?', 'h3': 'MISI', 'h4': 'OUR JOURNEY', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'WHY EMDEE?', 'h3': 'MISI', 'h4': 'OUR JOURNEY', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'OPERATIONAL HOUR', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'Solusi terbaik mengatasi permasalahan kulit wajahmu', 'h3': 'Moist Oil Free', 'source_url': 'https://emdeeclinic.com/'}
{'h1': 'OPERATIONAL HOUR', 'source_url': 'https://emdeeclinic.com/'}
{'h2': 'SEE OUR RESULT', 'h3': 'YOUR F

In [6]:
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

prompt = ChatPromptTemplate.from_template(template=template)
chain = prompt | model
response = chain.invoke({"question": user_query, "context": result})
print(response)


Emdee offers a range of skincare solutions to address various skin concerns. The context provided highlights several products and their features, such as "Solusi terbaik mengatasi permasalahan kulit wajahmu" (Purifying Wash) and "Solusi terbaik mengatasi permasalahan kulit wajahmu" (Purifying Cleanser). These products are designed to help users with their skin issues.
